In [1]:
import pickle
import json
import numpy
from glob import glob
import random

In [32]:
food_list={
    'apple': '/14_apple',
    'banana': '/03_banana',
    'carrot': '/04_carrot',
    'grape': '/18_grape',
    'cucumber': '/15_cucumber',
    'egg': '/08_egg',
    'eggplant': '/05_eggplant',
    'greenpepper': '/06_greenpepper',
    'pea': '/17_pea',
    'kiwi': '/10_kiwi',
    'lemon': '/09_lemon',
    'onion': '/13_onion',
    'orange': '/00_orange',
    'potatoes': '/12_potatoes',
    'bread': '/16_bread',
    'avocado': '/11_avocado',
    'strawberry': '/19_strawberry',
    'sweetpotato': '/02_sweetpotato',
    'tomato': '/07_tomato',
    'turnip': '/01_turnip'
    #'orange02': '/orange02'
}

# food_id contains 'foodname: pos' pairs
food_id={}
i=1
for key in food_list:
    food_id[key]=i
    i+=1

colors=['bl','br','wh']
numbers=['1','2','3']

speaker_division={}
speaker_division["train"]=[3, 5, 11, 19, 29, 30, 36, 38, 142]
speaker_division["val"]=[3, 5, 11, 19, 29, 30, 36, 38, 142]
speaker_division["test"]=[144, 151]

In [33]:
def get_image_info(image_path):
    info=image_path.split("/")[8]
    name=info.split("_")[0]
    if name=="orange":
        color=info.split("_")[1]
        number=info.split("_")[2]
    else:
        color=info.split("_")[1][0:-1]
        number=info.split("_")[1][-1]
    return name, color, number

def find_audio(img_path, mode_1, mode_2='unclean',form='wav'):
    food_name, color, number = get_image_info(img_path)
    basic_number=20*(food_id[food_name]-1)+2+(eval(number)-1)*6
    # id_1 and id_2 describes all information
    # id_3 only gives name with the right form
    audio_base_path = "/net/papilio/storage2/yhaoyuan/LAbyLM/audios_trimmed_select"
    captions = []
    if color=='wh':
        id_1=str(basic_number+1)
        id_2=str(basic_number+2)
    elif color=='bl':
        id_1=str(basic_number+3)
        id_2=str(basic_number+4)
    elif color=='br':
        id_1=str(basic_number+5)
        id_2=str(basic_number+6)
    else:
        print(color)
        print(img_path)
        raise ValueError("Not recognized color")
    if number=='1':
        id_3=str(20*(food_id[food_name]-1)+1)
    else:
        id_3=str(20*(food_id[food_name]-1)+2)
    if mode_1 in ["train","val","test"]:
        captions_1 = [f'/{str(speaker)}/{id_1}.wav' for speaker in speaker_division[mode_1]]
        captions_2 = [f'/{str(speaker)}/{id_2}.wav' for speaker in speaker_division[mode_1]]
        # caption_1=audio_source_path+'/'+str(choice(speaker_division[mode_1]))+'/'+id_1+'.'+form
        # caption_2=audio_source_path+'/'+str(choice(speaker_division[mode_1]))+'/'+id_2+'.'+form
        # caption_3=audio_source_path+'/'+str(choice(speaker_division[mode_1]))+'/'+id_3+'.'+form
    else:
        raise ValueError("Mode not recognized. Please choose from: train, valid, test.")
    #description_1=glob.glob(audio_source_path+'/*/'+id_1+'.'+form)
    #description_2=glob.glob(audio_source_path+'/*/'+id_2+'.'+form)
    #description_3=glob.glob(audio_source_path+'/*/'+id_3+'.'+form)
    # caption_1=get_unit(caption_1)
    # caption_2=get_unit(caption_2)
    # caption_3=get_unit(caption_3)
    # if mode_2=='clean':
    #     return caption_1, caption_2
    # if mode_2=='unclean':
    #     return caption_3, caption_1, caption_2
    return captions_1 + captions_2

In [34]:
img_paths = "../../data/food_image_split.json"
with open(img_paths, "r") as f:
    img_paths = json.load(f)

audio_base_path = "/net/papilio/storage2/yhaoyuan/LAbyLM/audios_trimmed_select"
image_base_path = "/net/tateha/storage2/database/spolacq/FoodImagesA"

In [35]:
food_dataset = {}
food_dataset["audio_base_path"] = audio_base_path
food_dataset["image_base_path"] = image_base_path
food_dataset["data"] = {}

In [41]:
for mode in ["train","val","test"]:
    data_list = []
    for img in img_paths[mode]:
        data = {}
        data["image"] = img[len(image_base_path):]
        data["audio"] = find_audio(img, mode)
        data_list.append(data)
    food_dataset["data"][mode] = data_list

In [48]:
with open("food_dataset_VC.json", "w") as f:
    json.dump(food_dataset, f)

In [55]:
# check the split
import os
for data in food_dataset["data"]["test"]:
    assert os.path.isfile(food_dataset["image_base_path"]+data["image"]), "Not a valid audio file"
    assert os.path.isfile(food_dataset["audio_base_path"]+data["audio"][0]), "Not a valid audio file"